### NicheNet on a GeneSet of Interest  

In [1]:
.libPaths("/home/jupyter/libb") # add this path 
library(ArchR)
library(parallel)
library(H5weaver)
library(tidyverse)
library(dplyr)
library(hise)
library(GenomicFeatures)


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

In [2]:
source('/home/jupyter/NicheNet/scATAC_Supplements/Enrichment_Analysis.R')

In [4]:
# Load Data

In [57]:
enrichMotifs_all_PI <- readRDS('enrichMotifs_all_PI.RData')
MotifEnrichment <- enrichMotifs_all_PI
columnWithMotifs = "feature"
int <- enrichMotifs_all_PI
MotifEnrichment <- lapply(assays(enrichMotifs_all_PI),as.data.frame) %>% do.call('cbind',.) 
colnames(MotifEnrichment) <- names(assays(int))
#MotifEnrichment <- as.tbl(MotifEnrichment)
MotifEnrichment <- MotifEnrichment %>% group_by(.data[[ columnWithMotifs ]]) %>%
                        summarize(mlog10Padj = max(mlog10Padj)) 
head(MotifEnrichment)

feature,mlog10Padj
<chr>,<dbl>
AC0021266_638,41.440757
AC0125311_531,11.886557
AC2261502_577,7.198457
AHR_28,0.000000
AHRR_17,0.000000
AIRE_736,7.101157


In [58]:
MotifEnrichment$feature <- gsub("_.","",MotifEnrichment$feature)
head(MotifEnrichment)

feature,mlog10Padj
<chr>,<dbl>
AC002126638,41.440757
AC012531131,11.886557
AC226150277,7.198457
AHR8,0.000000
AHRR7,0.000000
AIRE36,7.101157


In [59]:
filteredTF <- ligandtf[rownames(ligandtf) %in% unique(MotifEnrichment$feature), ]      

In [55]:
head(ligandtf)

,CXCL1,CXCL2,CXCL3,CXCL5,PPBP,CXCL6,CXCL8,CXCL9,CXCL10,CXCL11,⋯,PCDHGB1,CDH6,CDH9,CDH12,CDH18,PCDHB16,PCDHA7,PCDHA6,PCDHA4,PCDHA12
A1BG,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
A1BG-AS1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
A2M-AS1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
A2ML1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [61]:
dim(MotifEnrichment)

[1] 870   2

In [63]:
 head(MotifEnrichment)

feature,mlog10Padj
<chr>,<dbl>
AC002126638,41.440757
AC012531131,11.886557
AC226150277,7.198457
AHR8,0.000000
AHRR7,0.000000
AIRE36,7.101157


In [64]:
test_enrich <- MotifSetEnrichmentAnalysis(ligandtf,MotifEnrichment,columnWithMotifs = 'feature',
                                          ligands = colnames(filteredTF)[colSums(filteredTF) > 0],
                                          mlogPval_threshold = 2,
                                          annotationName = 'CellType', annotation = "none",
                                          numCores = 30, verbose = FALSE)

In [65]:
head(test_enrich)

,ligand,p_val,adjp_val,PercentSigTF,PercInNicheNet,CellType
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
CXCL1,CXCL1,0.4444444,1,0,0,none
CXCL2,CXCL2,0.4444444,1,0,0,none
CXCL3,CXCL3,0.4444444,1,0,0,none
CXCL8,CXCL8,0.4444444,1,0,0,none
CXCL10,CXCL10,0.4444444,1,0,0,none
XCL1,XCL1,0.4444444,1,0,0,none


In [67]:
range(test_enrich$adjp_val)

[1] 3.873475e-321  1.000000e+00

In [68]:
min(test_enrich$adjp_val)

[1] 3.873475e-321

In [69]:
max(test_enrich$adjp_val)

[1] 1